In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout,  LSTM, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [78]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
13087,5,"['G', 'U']",False,['Enchantment'],['none'],Cumulative upkeep {1} (At the beginning of you...,['Cumulative upkeep'],0,0,rare,True,1042,True,0.66
2947,3,['G'],False,['Sorcery'],['none'],Bolster 3. (Choose a creature with the least t...,['Bolster'],0,0,uncommon,False,7723,True,0.04
18513,3,['G'],False,['Creature'],"['Satyr', 'Rogue']",{3}{G}: Target creature must be blocked this t...,['no keywords'],2,1,uncommon,False,7240,True,0.02
22407,2,['B'],False,['Artifact'],['Equipment'],"When Thieves' Tools enters the battlefield, cr...","['Treasure', 'Equip']",0,0,common,False,10418,True,0.07
3595,3,['C'],False,['Artifact'],['none'],"{2}: Until end of turn, Chimeric Sphere become...",['no keywords'],0,0,uncommon,False,1286,True,0.19


In [79]:
no_prices_df = df[df['usd'].isna()]

In [80]:
df.drop(df[df['usd'].isna()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [81]:
df['colors'] = df['colors'].apply(literal_eval)

In [82]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


#### Text processing

In [83]:
df['tokenized_text']=df['oracle_text'].apply(word_tokenize)

In [84]:
text_model = Word2Vec(sentences=df['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)

In [85]:
def get_sentence_embedding(tokens):
    tokens = [token for token in tokens if token in text_model.wv.key_to_index]
    if tokens:
        return sum(text_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None

In [86]:
df['text_embedding'] = df['oracle_text'].apply(get_sentence_embedding)

In [87]:
df['text_embedding']

0        [-0.51127124, 0.7780097, -0.035495322, 1.00399...
1        [-1.2777522, 0.23359896, 0.013131049, 1.333495...
2        [-0.9803241, 0.21962857, 0.011534677, 0.834389...
3        [-0.91731036, 0.734758, -0.036130466, 1.140536...
4        [-1.4610449, 0.13186991, 0.09089973, 1.152816,...
                               ...                        
25484    [-1.1082821, 0.21531929, -0.018388567, 1.04077...
25485    [-1.5607055, 0.3205959, 0.100274175, 1.5218605...
25486    [-1.1958561, 0.19678542, 0.036566474, 1.186480...
25487    [-1.2305026, 0.20982346, -0.03580578, 1.151475...
25488    [-1.0753328, 0.41487616, 0.013102834, 1.054587...
Name: text_embedding, Length: 25489, dtype: object

In [88]:
df['tokenized_types']=df['types'].apply(word_tokenize)
types_model = Word2Vec(sentences=df['types'], vector_size=100, window=5, min_count=1, workers=4)
def get_types_embedding(tokens):
    tokens = [token for token in tokens if token in types_model.wv.key_to_index]
    if tokens:
        return sum(types_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None
df['types_embedding'] = df['types'].apply(get_types_embedding)
df['types_embedding']

0        [0.14906012, -0.21770309, 0.05100936, 0.053071...
1        [-0.20235693, -0.052378323, -0.3317952, -0.071...
2        [-0.20235693, -0.052378323, -0.3317952, -0.071...
3        [0.43364996, -0.19831982, 0.035760514, -0.4332...
4        [0.14906012, -0.21770309, 0.05100936, 0.053071...
                               ...                        
25484    [-0.20235693, -0.052378323, -0.3317952, -0.071...
25485    [0.48035628, -0.19263104, -0.061860748, -0.425...
25486    [-0.20235693, -0.052378323, -0.3317952, -0.071...
25487    [-0.20235693, -0.052378323, -0.3317952, -0.071...
25488    [-0.20235693, -0.052378323, -0.3317952, -0.071...
Name: types_embedding, Length: 25489, dtype: object

In [89]:
df['tokenized_subtypes']=df['sub_types'].apply(word_tokenize)
subtypes_model = Word2Vec(sentences=df['sub_types'], vector_size=100, window=5, min_count=1, workers=4)
def get_types_embedding(tokens):
    tokens = [token for token in tokens if token in subtypes_model.wv.key_to_index]
    if tokens:
        return sum(subtypes_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None
df['subtypes_embedding'] = df['sub_types'].apply(get_types_embedding)
df['subtypes_embedding']

0        [-0.37045395, 0.18439247, 0.06258764, -0.54133...
1        [-0.27447513, -0.0665222, 0.20202424, -0.36353...
2        [-0.10581706, -0.22339688, 0.15656523, -0.4000...
3        [-0.21246275, 0.009257782, 0.06773925, -0.2428...
4        [-0.21246275, 0.009257782, 0.06773925, -0.2428...
                               ...                        
25484    [-0.2094457, -0.18556929, 0.25844848, -0.32577...
25485    [-0.21246275, 0.009257782, 0.06773925, -0.2428...
25486    [-0.4249401, -0.07015147, 0.1792006, -0.378282...
25487    [-0.2707535, -0.038897317, 0.19458976, -0.2887...
25488    [-0.2767891, -0.08932997, 0.13954143, -0.48634...
Name: subtypes_embedding, Length: 25489, dtype: object

In [90]:
df['tokenized_keywords']=df['keywords'].apply(word_tokenize)
keywords_model = Word2Vec(sentences=df['keywords'], vector_size=100, window=5, min_count=1, workers=4)
def get_types_embedding(tokens):
    tokens = [token for token in tokens if token in keywords_model.wv.key_to_index]
    if tokens:
        return sum(keywords_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None
df['keywords_embedding'] = df['keywords'].apply(get_types_embedding)
df['keywords_embedding']

0        [-0.0070120296, -0.12131895, 0.020730816, -0.2...
1        [0.09644347, -0.18145148, -0.3692048, 0.109952...
2        [0.12041464, -0.14207205, -0.083954014, 0.0695...
3        [-0.14961502, 0.1220414, -0.1562332, 0.2643179...
4        [-0.14961502, 0.1220414, -0.1562332, 0.2643179...
                               ...                        
25484    [-0.086470515, -0.13292374, -0.059534166, -0.0...
25485    [-0.14961502, 0.1220414, -0.1562332, 0.2643179...
25486    [0.09644347, -0.18145148, -0.3692048, 0.109952...
25487    [-0.14961502, 0.1220414, -0.1562332, 0.2643179...
25488    [-0.09700499, -0.010879397, -0.1195374, 0.1226...
Name: keywords_embedding, Length: 25489, dtype: object

In [91]:
df.columns

Index(['cmc', 'types', 'sub_types', 'oracle_text', 'keywords', 'power',
       'toughness', 'rarity', 'released_at', 'usd', 'B', 'C', 'G', 'N', 'R',
       'U', 'W', '_legendary', '_booster', '_resrved', 'tokenized_text',
       'text_embedding', 'tokenized_types', 'types_embedding',
       'tokenized_subtypes', 'subtypes_embedding', 'tokenized_keywords',
       'keywords_embedding'],
      dtype='object')

In [92]:
df.drop(columns=['sub_types','types', 'keywords', 'oracle_text',
        'tokenized_types','tokenized_subtypes', 'tokenized_keywords',
        'tokenized_text'],  inplace=True)

#### The Pipeline and neural network

In [93]:
df.head(3)

,cmc,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved,text_embedding,types_embedding,subtypes_embedding,keywords_embedding
0,2,0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0,"[-0.51127124, 0.7780097, -0.035495322, 1.00399...","[0.14906012, -0.21770309, 0.05100936, 0.053071...","[-0.37045395, 0.18439247, 0.06258764, -0.54133...","[-0.0070120296, -0.12131895, 0.020730816, -0.2..."
1,4,1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0,"[-1.2777522, 0.23359896, 0.013131049, 1.333495...","[-0.20235693, -0.052378323, -0.3317952, -0.071...","[-0.27447513, -0.0665222, 0.20202424, -0.36353...","[0.09644347, -0.18145148, -0.3692048, 0.109952..."
2,5,5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0,"[-0.9803241, 0.21962857, 0.011534677, 0.834389...","[-0.20235693, -0.052378323, -0.3317952, -0.071...","[-0.10581706, -0.22339688, 0.15656523, -0.4000...","[0.12041464, -0.14207205, -0.083954014, 0.0695..."


In [94]:
X = df.drop(columns='usd')
y = df['usd']

In [95]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [96]:
X_preprocessed= pipeline.fit_transform(X)

In [97]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__text_embedding', 'remainder__types_embedding',
       'remainder__subtypes_embedding', 'remainder__keywords_embedding'],
      dtype=object)

In [98]:
processed_df = pd.DataFrame(X_preprocessed, columns=column_names)

In [99]:
processed_df.head(3)

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,...,remainder__R,remainder__U,remainder__W,remainder___legendary,remainder___booster,remainder___resrved,remainder__text_embedding,remainder__types_embedding,remainder__subtypes_embedding,remainder__keywords_embedding
0,-0.787786,-0.818064,0.83164,-0.74473,1.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,1,0,"[-0.51127124, 0.7780097, -0.035495322, 1.00399...","[0.14906012, -0.21770309, 0.05100936, 0.053071...","[-0.37045395, 0.18439247, 0.06258764, -0.54133...","[-0.0070120296, -0.12131895, 0.020730816, -0.2..."
1,-0.237532,1.302998,0.930864,0.397826,1.0,0.0,0.0,0.0,0.0,0,...,0,1,0,0,1,0,"[-1.2777522, 0.23359896, 0.013131049, 1.333495...","[-0.20235693, -0.052378323, -0.3317952, -0.071...","[-0.27447513, -0.0665222, 0.20202424, -0.36353...","[0.09644347, -0.18145148, -0.3692048, 0.109952..."
2,1.963486,1.833263,0.967534,0.969104,0.0,1.0,0.0,0.0,0.0,1,...,1,1,0,1,0,0,"[-0.9803241, 0.21962857, 0.011534677, 0.834389...","[-0.20235693, -0.052378323, -0.3317952, -0.071...","[-0.10581706, -0.22339688, 0.15656523, -0.4000...","[0.12041464, -0.14207205, -0.083954014, 0.0695..."


In [100]:
processed_df =processed_df.astype({
    'num__power': float,
    'num__toughness': float,
    'num__released_at': float,
    'num__cmc': float
})

In [101]:
processed_df = processed_df.astype({
    "cat__rarity_common": int,
    "cat__rarity_mythic": int,
    "cat__rarity_rare": int,
    "cat__rarity_special": int,
    "cat__rarity_uncommon": int,
    "remainder__B": int,
    "remainder__C": int,
    "remainder__G": int,
    "remainder__N": int,
    "remainder__R": int,
    "remainder__U": int,
    "remainder__W": int,
    "remainder___legendary": int,
    "remainder___booster": int,
    "remainder___resrved": int
})

In [102]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   num__power                     25489 non-null  float64
 1   num__toughness                 25489 non-null  float64
 2   num__released_at               25489 non-null  float64
 3   num__cmc                       25489 non-null  float64
 4   cat__rarity_common             25489 non-null  int64  
 5   cat__rarity_mythic             25489 non-null  int64  
 6   cat__rarity_rare               25489 non-null  int64  
 7   cat__rarity_special            25489 non-null  int64  
 8   cat__rarity_uncommon           25489 non-null  int64  
 9   remainder__B                   25489 non-null  int64  
 10  remainder__C                   25489 non-null  int64  
 11  remainder__G                   25489 non-null  int64  
 12  remainder__N                   25489 non-null 

In [103]:
indexes = processed_df[processed_df['remainder__text_embedding'].isna()].index

In [104]:
processed_df.drop(indexes, inplace=True)
processed_df.reset_index(drop=True, inplace=True)

In [117]:
y.drop(indexes, inplace=True)
y.reset_index(drop=True, inplace=True)

In [118]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25380 entries, 0 to 25379
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   num__power                     25380 non-null  float64
 1   num__toughness                 25380 non-null  float64
 2   num__released_at               25380 non-null  float64
 3   num__cmc                       25380 non-null  float64
 4   cat__rarity_common             25380 non-null  int64  
 5   cat__rarity_mythic             25380 non-null  int64  
 6   cat__rarity_rare               25380 non-null  int64  
 7   cat__rarity_special            25380 non-null  int64  
 8   cat__rarity_uncommon           25380 non-null  int64  
 9   remainder__B                   25380 non-null  int64  
 10  remainder__C                   25380 non-null  int64  
 11  remainder__G                   25380 non-null  int64  
 12  remainder__N                   25380 non-null 

In [119]:
X_numerical = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc']]
X_categorical = processed_df[['remainder___legendary', 'remainder___booster', 'remainder___resrved',
                              'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare', 'cat__rarity_special', 'cat__rarity_uncommon',
                              'remainder__B', 'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R', 'remainder__U', 'remainder__W']]
X_text = processed_df[['remainder__text_embedding']]
X_subtypes= processed_df[['remainder__subtypes_embedding']]
X_types = processed_df[['remainder__types_embedding']]
X_keys = processed_df[['remainder__keywords_embedding']]

In [120]:
X_numerical.shape, X_categorical.shape, X_text.shape, X_subtypes.shape, X_types.shape, X_keys.shape

((25380, 4), (25380, 15), (25380, 1), (25380, 1), (25380, 1), (25380, 1))

In [136]:
type(X_text['remainder__text_embedding'][0])

numpy.ndarray

In [144]:
X_text['remainder__text_embedding'] = X_text['remainder__text_embedding'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) or isinstance(x, np.ndarray) else x)

/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_12472/1709872702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_text['remainder__text_embedding'] = X_text['remainder__text_embedding'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) or isinstance(x, np.ndarray) else x)


In [146]:
X_text = tf.convert_to_tensor(X_text['remainder__text_embedding'].values, dtype=tf.float32)

ValueError: setting an array element with a sequence.

In [123]:
X_numerical_train, X_numerical_test, X_categorical_train, X_categorical_test, X_text_train, X_text_test,X_subtypes_train, X_subtypes_test, X_types_train, X_types_test, X_keys_train, X_keys_test, y_train, y_test = train_test_split(
    X_numerical, X_categorical, X_text, X_subtypes, X_types, X_keys, y, test_size=0.30, random_state=42
)

In [124]:
def initialize_model():

    model_numerical = Sequential([
        Input(shape=(4,), name='input_numerical'),
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu')
    ], name='model_numerical')

    model_categorical = Sequential([
        Input(shape=(15,), name='input_categorical'),
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu')
    ], name='model_categorical')

    model_text = Sequential([
        Input(shape=(300,), name='input_text'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
    ], name='model_text')


    model_type = Sequential([
        Input(shape=(100,), name='input_type'),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
    ], name='model_type')


    model_subtype = Sequential([
        Input(shape=(100,), name='input_subtype'),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_subtype')


    model_keys = Sequential([
        Input(shape=(100,), name='input_keys'),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_keys')


    concatenated = Concatenate()([
        model_numerical.output,
        model_categorical.output,
        model_text.output,
        model_type.output,
        model_keys.output,
        model_subtype.output
    ])

    output = Dense(1, activation='linear', name='output')(concatenated)


    model = models.Model(inputs=[model_numerical.input, model_categorical.input, model_text.input, model_type.input, model_keys.input, model_subtype.input], outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mse', 'accuracy'])

    return model


In [125]:
model = initialize_model()

In [126]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=False)

    train_data = {
        'input_numerical': X_numerical_train,
        'input_categorical': X_categorical_train,
        'input_text': X_text_train,
        'input_type': X_types_train,
        'input_subtype': X_subtypes_train,
        'input_keys': X_keys_train
    }

    test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }

    history = model.fit(
        train_data,
        y_train,
        epochs=20,
        batch_size=16,
        validation_data=(test_data, y_test),
        callbacks=[es],
        verbose=1
    )

    return history


In [127]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None, 300)]                0         []                            
                                                                                                  
 dropout_13 (Dropout)        (None, 300)                  0         ['input_text[0][0]']          
                                                                                                  
 input_numerical (InputLaye  [(None, 4)]                  0         []                            
 r)                                                                                               
                                                                                                  
 dense_32 (Dense)            (None, 256)                  77056     ['dropout_13[0][0]']    

In [128]:
get_history()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }
model.evaluate(test_data, y_test)

239/239 [==============================] - 0s 2ms/step - loss: 1885.5562 - mse: 1881.6256 - accuracy: 2.6154e-04


[1885.55615234375, 1881.6256103515625, 0.00026154046645388007]

In [ ]:
no_prices_df = df[df['usd'].isna()]